In [13]:
import psycopg2
from psycopg2 import sql
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

In [15]:
def get_db_connection():
    """Create and return a database connection"""
    try:
        conn = psycopg2.connect(
            host="localhost",
            database="fitness_tracker",
            user="postgres",  # Replace with your PostgreSQL username
            password="Badger2022!"  # Replace with your PostgreSQL password
        )
        print("✓ Database connection successful!")
        return conn
    except Exception as e:
        print(f"✗ Connection failed: {e}")
        return None

conn = get_db_connection()

✓ Database connection successful!


In [17]:
def list_tables(conn):
    """List all tables in the database"""
    query = """
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'public'
        ORDER BY table_name;
    """
    df = pd.read_sql_query(query, conn)
    return df

In [19]:
if conn:
    tables = list_tables(conn)
    print("Tables in database:")
    print(tables)

Tables in database:
  table_name
0  food_logs
1      foods
2      users


In [21]:
def describe_table(conn, table_name):
    """Show column information for a specific table"""
    query = """
        SELECT column_name, data_type, character_maximum_length
        FROM information_schema.columns
        WHERE table_name = %s
        ORDER BY ordinal_position;
    """
    df = pd.read_sql_query(query, conn, params=(table_name,))
    return df

In [25]:
# Cell: Check if data exists
def check_food_count(conn):
    """Check how many foods are in the database"""
    query = "SELECT COUNT(*) FROM foods;"
    cursor = conn.cursor()
    cursor.execute(query)
    count = cursor.fetchone()[0]
    cursor.close()
    return count

if conn:
    food_count = check_food_count(conn)
    print(f"Total foods in database: {food_count}")

Total foods in database: 13636


In [27]:
def check_table_structure(conn):
    """Directly check the foods table structure"""
    query = """
        SELECT column_name, data_type, character_maximum_length
        FROM information_schema.columns
        WHERE table_schema = 'public' AND table_name = 'foods'
        ORDER BY ordinal_position;
    """
    cursor = conn.cursor()
    cursor.execute(query)
    columns = cursor.fetchall()
    cursor.close()
    return columns

if conn:
    structure = check_table_structure(conn)
    print("Foods table structure:")
    for col in structure:
        print(f"  {col[0]}: {col[1]}")

Foods table structure:
  id: integer
  fdc_id: integer
  description: text
  calories_per_100g: numeric
  protein_per_100g: numeric
  carbs_per_100g: numeric
  fat_per_100g: numeric
  saturated_fat_per_100g: numeric
  fiber_per_100g: numeric
  sugar_per_100g: numeric
  sodium_per_100g: numeric
  cholesterol_per_100g: numeric
  vitamin_a_per_100g: numeric
  vitamin_c_per_100g: numeric
  vitamin_d_per_100g: numeric
  calcium_per_100g: numeric
  iron_per_100g: numeric
  potassium_per_100g: numeric
  serving_size: numeric
  serving_unit: text
  serving_description: text


In [29]:
# Convert entire foods table to DataFrame
df = pd.read_sql_query("SELECT * FROM foods;", conn)

# View the DataFrame
print(df.head())  # First 5 rows
print(df.shape)   # (rows, columns)
print(df.columns) # Column names

    id  fdc_id                                        description  \
0  320  167512  Pillsbury Golden Layer Buttermilk Biscuits, Ar...   
1  321  167513  Pillsbury, Cinnamon Rolls with Icing, refriger...   
2  322  167514  Kraft Foods, Shake N Bake Original Recipe, Coa...   
3  323  167515     George Weston Bakeries, Thomas English Muffins   
4  324  167516         Waffles, buttermilk, frozen, ready-to-heat   

   calories_per_100g  protein_per_100g  carbs_per_100g  fat_per_100g  \
0               5.88             13.24           41.18         307.0   
1               4.34             11.27           53.42         330.0   
2               6.10              3.70           79.80         377.0   
3               8.00              1.80           46.00         232.0   
4               6.58              9.22           41.05         273.0   

   saturated_fat_per_100g  fiber_per_100g  sugar_per_100g  ...  \
0                     NaN             1.2             NaN  ...   
1                   

In [31]:
df.head()


,id,fdc_id,description,calories_per_100g,protein_per_100g,carbs_per_100g,fat_per_100g,saturated_fat_per_100g,fiber_per_100g,sugar_per_100g,...,cholesterol_per_100g,vitamin_a_per_100g,vitamin_c_per_100g,vitamin_d_per_100g,calcium_per_100g,iron_per_100g,potassium_per_100g,serving_size,serving_unit,serving_description
0,320,167512,"Pillsbury Golden Layer Buttermilk Biscuits, Ar...",5.88,13.24,41.18,307.0,NaN,1.2,NaN,...,NaN,1059.0,NaN,NaN,NaN,0.0,2.94,34.0,serving,None
1,321,167513,"Pillsbury, Cinnamon Rolls with Icing, refriger...",4.34,11.27,53.42,330.0,NaN,1.4,28.0,...,NaN,780.0,0.0,NaN,0.1,0.0,3.25,44.0,serving 1 roll with icing,None
2,322,167514,"Kraft Foods, Shake N Bake Original Recipe, Coa...",6.10,3.70,79.80,377.0,NaN,NaN,NaN,...,NaN,2182.0,NaN,NaN,NaN,NaN,NaN,28.0,serving,None
3,323,167515,"George Weston Bakeries, Thomas English Muffins",8.00,1.80,46.00,232.0,NaN,NaN,180.0,...,NaN,345.0,0.0,NaN,NaN,NaN,0.31,57.0,serving,None
4,324,167516,"Waffles, buttermilk, frozen, ready-to-heat",6.58,9.22,41.05,273.0,NaN,2.2,279.0,...,126.0,621.0,401.0,NaN,0.0,15.0,1.90,39.0,"waffle, square",None


In [45]:
# Create a new column with protein per calorie
df['protein_per_calorie'] = df['protein_per_100g'] / df['calories_per_100g']

# Drop rows with NaN values and sort
df_clean = df.dropna(subset=['protein_per_calorie'])  # dropna() not drop_na()
df_clean.sort_values('protein_per_calorie', ascending=False).head(20)  # Add .head() to see results


,id,fdc_id,description,calories_per_100g,protein_per_100g,carbs_per_100g,fat_per_100g,saturated_fat_per_100g,fiber_per_100g,sugar_per_100g,...,vitamin_a_per_100g,vitamin_c_per_100g,vitamin_d_per_100g,calcium_per_100g,iron_per_100g,potassium_per_100g,serving_size,serving_unit,serving_description,protein_per_calorie
7351,7665,174857,"Beverages, MOTTS, Apple juice light, fortified...",0.0,0.1,5.10,22.0,NaN,0.0,3.0,...,13.0,NaN,NaN,24.0,0.0,0.00,NaN,None,None,inf
12948,13268,2710180,"Vegetable oil, NFS",0.0,100.0,0.00,900.0,NaN,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,13.37,224.0,10205,None,inf
12950,13270,2710182,Coconut oil,0.0,99.1,0.84,895.0,NaN,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,82.50,224.0,10205,None,inf
12951,13271,2710183,Corn oil,0.0,100.0,0.00,900.0,NaN,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,13.40,224.0,10205,None,inf
12952,13272,2710184,Cottonseed oil,0.0,100.0,0.00,884.0,NaN,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,25.90,224.0,10205,None,inf
12954,13274,2710186,Olive oil,0.0,100.0,0.00,900.0,NaN,0.0,1.0,...,2.0,0.0,0.0,0.0,0.0,15.52,224.0,10205,None,inf
12955,13275,2710187,Peanut oil,0.0,100.0,0.00,900.0,NaN,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,16.20,224.0,10205,None,inf
12956,13276,2710188,Canola oil,0.0,100.0,0.00,900.0,NaN,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,6.61,224.0,10205,None,inf
12957,13277,2710189,Safflower oil,0.0,100.0,0.00,900.0,NaN,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7.65,224.0,10205,None,inf
12958,13278,2710190,Sesame oil,0.0,100.0,0.00,884.0,NaN,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,14.20,224.0,10205,None,inf
